<a href="https://colab.research.google.com/github/TayJen/H_Tatarsan2022/blob/main/H_Tatarstan2022_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Импорт библиотек

In [1]:
import os
import pandas as pd
import numpy as np

np.random.seed(59)

### Пример решения

In [2]:
path_sample = '/content/drive/MyDrive/H_Tatarstan2022/data/sample_solution.csv'
df_sample = pd.read_csv(path_sample, sep=';')
df_sample.head(15)

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_05-31-51_AM.jpg,3,0.825893,0.035938,0.053646,0.305556,0.373148
1,25-11-2021_06-10-03_AM.jpg,3,0.917864,0.718750,0.763392,0.169737,0.357895
2,24-11-2021_09-05-00_AM.jpg,3,0.536900,0.571484,0.587500,0.320139,0.389583
3,24-11-2021_09-05-00_AM.jpg,3,0.712740,0.873438,0.892578,0.313195,0.398611
4,24-11-2021_09-05-00_AM.jpg,3,0.770197,0.895703,0.915235,0.325695,0.408333
5,24-11-2021_09-05-00_AM.jpg,3,0.786203,0.682031,0.700391,0.306945,0.395833
6,24-11-2021_09-05-00_AM.jpg,3,0.829074,0.642969,0.670313,0.422222,0.554166
7,24-11-2021_02-42-46_PM.jpg,3,0.529614,0.328497,0.347471,0.144737,0.223027
8,24-11-2021_02-42-46_PM.jpg,3,0.827705,0.313244,0.332218,0.136184,0.226974
9,23-11-2021_05-36-22_PM.jpg,3,0.872894,0.205729,0.242709,0.263889,0.365741


## Подготовка данных

In [3]:
%cd /content/drive/MyDrive/H_Tatarstan2022/
!pwd

/content/drive/MyDrive/H_Tatarstan2022
/content/drive/MyDrive/H_Tatarstan2022


In [4]:
# !unzip data/zipped/train_dataset_train.zip -d data/
# !unzip data/zipped/test_dataset_test.zip -d data/

In [5]:
len(os.listdir('data/train/images')), len(os.listdir('data/test/images'))

(2002, 855)

In [6]:
# Распределение объектов в train
pd.Series([file_name.split('_')[-1]
           for file_name in os.listdir('data/train/labels')]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [7]:
# Создадим обучающий датасет в формате yolo
# !mkdir data_for_yolo
# !mkdir data_for_yolo/data
# !mkdir data_for_yolo/data/images
# !mkdir data_for_yolo/data/labels
# !mkdir data_for_yolo/data/images/train
# !mkdir data_for_yolo/data/labels/train
# !mkdir data_for_yolo/data/images/test
# !mkdir data_for_yolo/data/labels/test

In [8]:
import yaml

yaml_content = """
train: data_for_yolo/data/images/train/
val: data_for_yolo/data/images/test/

# number of classes
nc: 5

# class names
names: ['human', 'head', 'face', 'car', 'carplate']
"""

with open('data_for_yolo/dataset.yaml', 'w') as f_yml:
    yaml.dump(yaml_content, f_yml)

In [9]:
# !mkdir data/train/yolo_labels

In [23]:
# Сделаем разметку как в Yolo, после этого нужным образом перемешаем и
# закинем в папку data_for_yolo
name_to_class_num = {
    'human': 0,
    'head': 1,
    'face': 2,
    'car': 3,
    'carplate': 4
}

for img_name in os.listdir('data/train/images'):
    print(img_name.split('.')[0])
    break

25-11-2021_06-28-29_AM


In [24]:
for img_name in os.listdir('data/train/images/'):
    img_name = img_name.split('.')[0]
    print(img_name)
    for f in os.listdir('data/train/labels/'):
        if f.startswith(img_name):
            print(f)
    break

25-11-2021_06-28-29_AM
25-11-2021_06-28-29_AM_human..txt
25-11-2021_06-28-29_AM_head..txt
25-11-2021_06-28-29_AM_face..txt
25-11-2021_06-28-29_AM_car..txt


In [26]:
for img_name in os.listdir('data/train/images/'):
    img_name = img_name.split('.')[0]
    print(img_name)
    for f in os.listdir('data/train/labels/'):
        if f.startswith(img_name):
            print(f.split('.')[0].split('_')[-1])
            with open('data/train/labels/' + f, 'r') as f_old:
                print(f_old.readlines())
    break

25-11-2021_06-28-29_AM
human
['0 0.5355512152777778 0.7598996913580247 0.08703124999999999 0.34498456790123466\n', '0 0.6109288194444444 0.809579475308642 0.06459201388888886 0.2568132716049384\n']
head
['0 0.6060763888888889 0.7109413580246914 0.027326388888888904 0.05478395061728395\n', '0 0.5411002604166666 0.6192052469135803 0.02878038194444442 0.060462962962962975\n']
face
['0 0.6046875 0.7285956790123457 0.017083333333333388 0.023487654320987695\n', '0 0.5391970486111111 0.6480131172839506 0.018220486111111118 0.03158179012345675\n']
car
['0 0.08944661458333333 0.24220679012345678 0.17889322916666667 0.2080246913580247\n', '0 0.15489149305555555 0.0732716049382716 0.14311631944444445 0.11594135802469135\n']


In [ ]:
old_path = 'data/train/labels/'
new_path = 'data/train/yolo_labels/'

for img_name in os.listdir('data/train/images/'):
    img_name = img_name.split('.')[0]
    print(img_name)
    with open(new_path + img_name + '.txt', 'w') as f_new:
        for f in os.listdir(old_path):
            if f.startswith(img_name):
                with open(old_path + f, 'r') as f_old:
                    f_new.write(f_old.readlines())
    break

## Model

In [ ]:
# !git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 12547, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 12547 (delta 49), reused 42 (delta 19), pack-reused 12452
Receiving objects: 100% (12547/12547), 12.53 MiB | 3.82 MiB/s, done.
Resolving deltas: 100% (8584/8584), done.


In [ ]:
# !python yolov5/train.py --freeze 10